# ELT Project

## Finding Data:

DATA SOURCE: https://www.kaggle.com/datasnaek/youtube-new/data <br/>
Utilizing: <br/>
3 csv files with Video Information (Canada, US, and Britain) <br/>
3 json files with Category Assignment (Canada, US, and Britain) <br/>

## Data Cleanup & Analysis

Plan and document the following:
* The sources of data that you will extract from.
* The type of transformation needed for this data (cleaning, joining, filtering, aggregating, etc).
* The type of final production database to load the data into (relational or non-relational).
* The final tables or collections that will be used in the production database.

You will be required to submit a final technical report with the above information and steps required to reproduce your ETL process.

## Project Report:

Submit a Final Report that describes the following:
* Extract: your original data sources and how the data was formatted (CSV, JSON, pgAdmin 4, etc).
* Transform: what data cleaning or transformation was required.
* Load: the final database, tables/collections, and why this was chosen.

Please upload the report to Github and submit a link to Bootcampspot.

In [2]:
import os
import pandas as pd
import json
import requests
import numpy as np
import datetime as dt
# pd.options.display.max_rows = 3000

from pandas.io.json import json_normalize
from sqlalchemy import create_engine

# EXTRACT

In [3]:
#reading in Canada csv video info
ca_file = os.path.join("data", "CAvideos.csv")
CA_df = pd.read_csv(ca_file)

In [4]:
#reading in Canada json category keys
json_CA = os.path.join("data", "CA_category_id.json")
category_CA_df = pd.read_json(json_CA)

In [5]:
#reading in Great Britain csv video info
gb_file = os.path.join("data", "GBvideos.csv")
GB_df = pd.read_csv(gb_file)

In [6]:
#reading in Great Britain json category keys
json_GB = os.path.join("data", "GB_category_id.json")
category_GB_df = pd.read_json(json_GB)

In [7]:
#reading in United States csv video info
us_file = os.path.join("data", "USvideos.csv")
US_df = pd.read_csv(us_file)

In [8]:
#reading in United States json category keys
json_US = os.path.join("data", "US_category_id.json")
category_US_df = pd.read_json(json_US)

# TRANSFORM

In [9]:
#clean up Canada category df with json_normalize (pulls dictionary items into their own column)
#drop static YouTube info not needed for video database, rename column and cast category_id to number for merge
CA_category_df = json_normalize(category_CA_df['items'])
CA_category_df.drop(['etag', 'kind', 'snippet.assignable', 'snippet.channelId'], axis=1, inplace=True)
CA_category_df.rename(columns={'id': 'category_id', 'snippet.title': 'category_name'}, inplace=True)
CA_categories = CA_category_df.astype({'category_id': 'int64'})

In [10]:
#adding a column to Canada video df to define which country info came from after upcoming concat
CA_df.insert(1,"country", "CA") 

In [11]:
#merge category_names into CA_df
CAfull_df = CA_df.merge(CA_categories, how='left', on="category_id")

In [12]:
# CAfull_df.count()     # video_id: 40881 - category_name: 40807  [NaN: 74]

In [13]:
#clean up Great Britain category df with json_normalize (pulls dictionary items into their own column)
#drop static YouTube info not needed for video database, rename column and cast category_id to number for merge
GB_category_df = json_normalize(category_GB_df['items'])
GB_category_df.drop(['etag', 'kind', 'snippet.assignable', 'snippet.channelId'], axis=1, inplace=True)
GB_category_df.rename(columns={'id': 'category_id', 'snippet.title': 'category_name'}, inplace=True)
GB_categories = GB_category_df.astype({'category_id': 'int64'})

In [14]:
#adding a column to Great Britain video df to define which country info came from after upcoming concat
GB_df.insert(1,"country", "GB")

In [15]:
#merge category_names into GB_df
GBfull_df = GB_df.merge(GB_categories, how='left', on="category_id")

In [16]:
# GBfull_df.count()     # video_id: 38916 - category_name: 38826  [NaN: 90]

In [17]:
#clean up United States category df with json_normalize (pulls dictionary items into their own column)
#drop static YouTube info not needed for video database, rename column and cast category_id to number for merge
US_category_df = json_normalize(category_US_df['items'])
US_category_df.drop(['etag', 'kind', 'snippet.assignable', 'snippet.channelId'], axis=1, inplace=True)
US_category_df.rename(columns={'id': 'category_id', 'snippet.title': 'category_name'}, inplace=True)
US_categories = US_category_df.astype({'category_id': 'int64'})

In [18]:
#adding a column to United States video df to define which country info came from after upcoming concat
US_df.insert(1,"country", "US")

In [19]:
#merge category_names into US_df
USfull_df = US_df.merge(US_categories, how='left', on="category_id")

In [20]:
# USfull_df.count()     # video_id: 40949 - category_name: 40949  [NaN: 0]

In [21]:
#pull only the most recent stats per video (for each CA, GB, and US df)
#1> add MaxDate col: take most recent trending_date for each video_id and assign every copy of that video_id
#2> filter the df to only the videos where trending_date and MaxDate are the same

CAfull_df['MaxDate'] = CAfull_df.groupby('video_id').trending_date.transform('max') # CA_df COUNT: 40881(max)
final_CA_df = CAfull_df[CAfull_df['MaxDate'] == CAfull_df['trending_date']] # final_CA_df COUNT: 24427(max)

GBfull_df['MaxDate'] = GBfull_df.groupby('video_id').trending_date.transform('max') # GB_df COUNT: 38916(max)
final_GB_df = GBfull_df[GBfull_df['MaxDate'] == GBfull_df['trending_date']] # final_CA_df COUNT: 3300(max)

USfull_df['MaxDate'] = USfull_df.groupby('video_id').trending_date.transform('max') # US_df COUNT: 40949(max)
final_US_df = USfull_df[USfull_df['MaxDate'] == USfull_df['trending_date']] # final_US_df COUNT: 6354(max)

In [22]:
full_frame = pd.concat([final_CA_df, final_GB_df, final_US_df], ignore_index=True) # full_frame COUNT: 34081(max)

In [23]:
# FILL NaN VALUES: 'description' [filling 1116 NaN descriptions]
full_frame['description'].fillna("No description provided.", inplace=True)

In [24]:
# full_frame.count()     # video_id: 34081 - category_name: 34026  [NaN: 55 (others were dropped in the MaxDate filter)]

In [25]:
# FILL NaN VALUES: 'category_name'
# unsure why none of the json sets contained the category assigner for 'Nonprofits & Activism' (category_id 29 *Google)
# specifying this category_id for future use (if category_id != 29, will not fillna...)

In [26]:
# get index list of all category_id == 29 rows

# cat_id_29_idx = full_frame.index[full_frame['category_id'] == 29]
# cat_id_29_idx

In [27]:
#test frame based on category_id == 29
#run before and after setting np.array as series to replace NaN in category_name with 'Nonprofits...'

# validator = full_frame.loc[cat_id_29_idx]
# validator

In [28]:
#fill NaN 'Nonprofits...' in an array
cat_name_full_set = np.where(pd.isnull(full_frame.category_name), "Nonprofits & Activism", full_frame.category_name)

In [29]:
#convert array to pandas series and overwrite 'category_name' column
full_frame['category_name'] = pd.Series(cat_name_full_set)

In [30]:
#convert 'publish_time' to datetime format
full_frame['publish_time'] = pd.to_datetime(full_frame['publish_time'])

In [31]:
#pull just date from full publish_date
full_frame['publish_date'] = full_frame['publish_time'].dt.date

In [32]:
#drop unnecessary columns not using for project
full_frame.drop(['thumbnail_link', 
                 'ratings_disabled', 
                 'video_error_or_removed', 
                 'publish_time', 
                 'MaxDate'] , axis=1, inplace=True)

In [33]:
#reorganize columns
full_frame = full_frame[['video_id',
                         'title', 
                         'channel_title', 
                         'views', 
                         'likes', 
                         'dislikes', 
                         'comments_disabled', 
                         'comment_count', 
                         'description', 
                         'tags', 
                         'category_id', 
                         'category_name', 
                         'publish_date', 
                         'trending_date', 
                         'country']]

In [34]:
full_frame.head()

,video_id,title,channel_title,views,likes,dislikes,comments_disabled,comment_count,description,tags,category_id,category_name,publish_date,trending_date,country
0,0yIWz1XEeyc,Jake Paul Says Alissa Violet CHEATED with LOGA...,DramaAlert,1309699,103755,4613,False,12143,► Follow for News! - https://twitter.com/KEEMS...,"#DramaAlert|""Drama""|""Alert""|""DramaAlert""|""keem...",25,News & Politics,2017-11-13,17.14.11,CA
1,FyZMnhUtLfE,猎场 | Game Of Hunting 12【TV版】（胡歌、張嘉譯、祖峰等主演）,大劇獨播,158815,218,30,False,186,Thanks for watching the drama! Help more peopl...,"電視劇|""大陸電視劇""|""猎场""|""职场""|""商战""|""爱情""|""都市""|""胡歌""|""陈龙""...",1,Film & Animation,2017-11-12,17.14.11,CA
2,7MxiQ4v0EnE,Daang ( Full Video ) | Mankirt Aulakh | Sukh S...,Speed Records,5718766,127477,7134,False,8063,Song - Daang\nSinger - Mankirt Aulakh\nFaceboo...,"punjabi songs|""punjabi bhangra""|""punjabi music...",10,Music,2017-11-11,17.14.11,CA
3,gifPYwArCVQ,Fake Pet Smart Employee Prank!,NELK,557883,44558,621,False,9619,3 Days left to cop NELK merch: https://nelk.ca...,"prank|""pranks""|""nelk""|""nelkfilmz""|""nelkfilms""",23,Comedy,2017-11-13,17.14.11,CA
4,8NHA23f7LvU,Jason Momoa Wows Hugh Grant With Some Dothraki...,The Graham Norton Show,1496225,16116,236,False,605,I think Sarah Millican was very excited for th...,"Graham Norton|""Graham Norton Show Official""|""E...",24,Entertainment,2017-11-10,17.14.11,CA


# Load